In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
import statsmodels.formula.api as sm

In [45]:
train = pd.read_csv("data/2022_FH.csv")
train = train.dropna(subset = ["Game Prep"])
train.reset_index()

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_28526/92136705.py:1: DtypeWarning: Columns (59,63,67) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data/2022_FH.csv")


,index,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
0,0,1,Wednesday,W,UConn,B,Maddie Bacskai,GD-2,Training,Whole session,...,1.00,25.0,12.25,12.25,1.0,11.43%,2.59%,14.02%,1222.39,216
1,1,1,Wednesday,W,UConn,F,Bente Baekers,GD-2,Training,Whole session,...,1.00,52.0,25.25,25.25,1.0,10.25%,0.80%,11.06%,1123.52,216
2,2,1,Wednesday,W,UConn,M,Jordan Carr,GD-2,Training,Whole session,...,1.00,74.0,36.50,36.50,1.0,18.93%,1.18%,20.11%,1240.37,216
3,3,1,Wednesday,W,UConn,M,Regan Cornelius,GD-2,Training,Whole session,...,1.00,71.0,34.50,34.50,1.0,9.56%,0.20%,9.76%,1132.51,216
4,4,1,Wednesday,W,UConn,B,Shannon Daley,GD-2,Training,Whole session,...,1.00,87.0,42.75,42.75,1.0,8.42%,0.56%,8.99%,1150.49,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16820,22398,65,Sunday,L,NCAA_UNC,M,Chloe Relford,NCAA_UNC,Game,4th quarter,...,1.17,19.0,36.18,32.85,1.1,23.59%,16.36%,39.95%,132.31,22
16821,22399,65,Sunday,L,NCAA_UNC,GK,Annabel Skubisz,NCAA_UNC,Game,4th quarter,...,1.68,1.0,4.28,6.16,0.69,8.83%,0.00%,8.83%,128.66,22
16822,22400,65,Sunday,L,NCAA_UNC,M,Lila Slattery,NCAA_UNC,Game,4th quarter,...,0.87,1.0,19.60,19.29,1.02,0.61%,0.00%,0.61%,103.11,22
16823,22401,65,Sunday,L,NCAA_UNC,M,Lauren Wadas,NCAA_UNC,Game,4th quarter,...,0.88,22.0,27.36,27.92,0.98,24.81%,13.24%,38.05%,135.05,22


In [46]:
train.columns

Index(['Week', 'Day', 'W L', 'Game Prep', 'Position', 'Athlete',
       'Session Name', 'Session', 'Phase Name', 'Duration', 'Date', 'HR Min',
       'HR AVG', 'HR Max', 'HR Min Per', 'HR Avg Per', 'HR Max Per', 'HR 1',
       'HR 2', 'HR 3', 'HR 4', 'HR 5', 'Total Distance', 'Distance/min',
       'Max Speed', 'Avg Speed', 'Sprints', 'Speed 1', 'Speed 2', 'Speed 3',
       'Speed 4', 'Speed 5', 'Decel 4', 'Decel 3', 'Decel 2', 'Decel 1',
       'Accel 1', 'Accel 2', 'Accel 3', 'Accel 4', 'Calories', 'TLScore',
       'Cardio Load', 'Recovery', 'TL Percent', 'TLScore Acute',
       'TLScore Chronic', 'TLScore TSB', 'HR Sum 4-5', 'Acute HR Sum',
       'Chronic HR Sum', 'TSB HR Sum', 'TD Percent', 'Acute TD', 'Chronic TD',
       'TSB TD', 'HIA', 'Acute HIA', 'Chronic HIA', 'TSB HIA', 'HSD',
       'Acute HSD', 'Chronic HSD', 'TSB HSD', 'Decel Sum', 'Acute Decel',
       'Chronic Decel', 'TSB Decel', 'HR 4 Percent', 'HR 5 Percent',
       'Percent Above 80', 'TRIMP', 'Duration (min)'],


**Cleaning data**
- delete all rows that aren't preparing for a game
- only use fall data (August - Dec) - regular season data
- don't include HR% because practices are varying length - actual time is more important
- filter to only include "whole session"
- filter to players who played more than 800 minutes for the 2022 season
- filter out goalies

In [47]:
#train = train[["Week", "W L", "Game Prep", "Position", "Athlete", "Session Name", "Phase Name", "Date", "HR Min", "HR AVG", "HR Max", "HR 1", "HR 2", "HR 3", "HR 4", "HR 5", "Total Distance", "Sprints", "Cardio Load", "TLScore", "Duration (min)"]]

In [48]:
train["Date"] = pd.to_datetime(train['Date'])

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_28526/2694793490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Date"] = pd.to_datetime(train['Date'])


In [49]:
#only keeping whole session, dates during season, and field player data
train = train[(train["Phase Name"] == "Whole session") & (train["Date"] >= "2022-08-01") & (train["Date"] <= "2022-12-01") & (train["Position"] != "GK")]

In [50]:
train.Athlete.unique()

array(['Amelia Albers', 'Bente Baekers', 'Jordan Carr', 'Regan Cornelius',
       'Shannon Daley', 'Sophie Dix', 'Peyton Halsey', 'Lane Herbert',
       'Greta Hinke', 'Lauren Hunter', 'Aerin Krys', 'Ana Medina Garcia',
       'Magnolia Powers', 'Chloe Relford', 'Lila Slattery',
       'Sophie Thomas', 'Lauren Wadas', 'Madeleine Zimmer',
       'Maja Zivojnovic', 'Kayla Blas', 'Alia Marshall', 'Katie Jones'],
      dtype=object)

In [51]:
#filtering out data for only players with minutes above 800 minutes of playing time over the course of the season (total possible playing time around 1600 minutes)
high_min_players = ['Bente Baekers', 'Shannon Daley', 'Peyton Halsey', 'Ana Medina Garcia', 'Chloe Relford', 'Lauren Wadas','Madeleine Zimmer','Kayla Blas','Alia Marshall']
train = train.loc[train.Athlete.isin(high_min_players),:]
train

,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,Duration,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
13150,52,Wednesday,NaN,Iowa,F,Bente Baekers,GD-3,Training,Whole session,3:22:48,...,1.00,101.0,32.97,36.74,0.90,23.10%,13.78%,36.88%,1259.05,203
13153,52,Wednesday,NaN,Iowa,B,Shannon Daley,GD-3,Training,Whole session,3:22:48,...,1.00,141.0,33.16,36.50,0.91,17.27%,14.99%,32.26%,1233.70,203
13155,52,Wednesday,NaN,Iowa,M,Peyton Halsey,GD-3,Training,Whole session,3:22:48,...,1.00,65.0,26.11,28.82,0.91,4.92%,8.10%,13.03%,1402.70,203
13161,52,Wednesday,NaN,Iowa,M,Ana Medina Garcia,GD-3,Training,Whole session,3:22:48,...,1.00,110.0,26.75,30.93,0.86,21.54%,7.31%,28.85%,1225.25,203
13163,52,Wednesday,NaN,Iowa,M,Chloe Relford,GD-3,Training,Whole session,3:22:48,...,1.00,148.0,34.63,35.86,0.97,14.52%,20.36%,34.89%,1233.70,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,65,Sunday,L,NCAA_UNC,M,Alia Marshall,NCAA_UNC,Game,Whole session,2:26:00,...,0.97,109.0,30.74,31.71,0.97,37.04%,8.77%,45.81%,906.42,146
22261,65,Sunday,L,NCAA_UNC,M,Ana Medina Garcia,NCAA_UNC,Game,Whole session,2:26:00,...,0.95,111.0,29.15,27.79,1.05,32.74%,22.65%,55.39%,955.08,146
22262,65,Sunday,L,NCAA_UNC,M,Chloe Relford,NCAA_UNC,Game,Whole session,2:26:00,...,1.17,111.0,36.18,32.85,1.1,14.42%,22.09%,36.51%,900.33,146
22265,65,Sunday,L,NCAA_UNC,M,Lauren Wadas,NCAA_UNC,Game,Whole session,2:26:00,...,0.88,88.0,27.36,27.92,0.98,12.90%,8.63%,21.53%,851.67,146


In [52]:
train['W L'].isnull().sum()

40

In [53]:
# dropping all NA W_L values bc that game was a scrimmage
train = train.dropna()
train.head()

,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,Duration,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
13879,53,Thursday,W,Maine,F,Bente Baekers,GD-3,Training,Whole session,2:02:28,...,0.94,80.0,32.50,32.66,0.99,22.77%,7.09%,29.86%,699.08,122
13880,53,Thursday,W,Maine,B,Kayla Blas,GD-3,Training,Whole session,2:02:28,...,1.24,62.0,25.67,29.48,0.87,18.02%,0.00%,18.02%,653.16,122
13883,53,Thursday,W,Maine,B,Shannon Daley,GD-3,Training,Whole session,2:02:28,...,0.91,99.0,34.18,34.54,0.99,20.79%,6.76%,27.56%,683.77,122
13885,53,Thursday,W,Maine,M,Peyton Halsey,GD-3,Training,Whole session,2:02:28,...,1.14,65.0,29.62,28.14,1.05,14.02%,26.29%,40.31%,770.52,122
13892,53,Thursday,W,Maine,M,Alia Marshall,GD-3,Training,Whole session,2:02:28,...,1.07,57.0,25.48,26.50,0.96,16.59%,0.14%,16.73%,648.05,122


In [54]:
#adding a binary column for W and Loss
train.columns = train.columns.str.replace(' ', '_')
train['W_L_binary'] = train['W_L'].apply(lambda x: 0 if x=='L' else 1)
train.head()

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_28526/1939409511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['W_L_binary'] = train['W_L'].apply(lambda x: 0 if x=='L' else 1)


,Week,Day,W_L,Game_Prep,Position,Athlete,Session_Name,Session,Phase_Name,Duration,...,Decel_Sum,Acute_Decel,Chronic_Decel,TSB_Decel,HR_4_Percent,HR_5_Percent,Percent_Above_80,TRIMP,Duration_(min),W_L_binary
13879,53,Thursday,W,Maine,F,Bente Baekers,GD-3,Training,Whole session,2:02:28,...,80.0,32.50,32.66,0.99,22.77%,7.09%,29.86%,699.08,122,1
13880,53,Thursday,W,Maine,B,Kayla Blas,GD-3,Training,Whole session,2:02:28,...,62.0,25.67,29.48,0.87,18.02%,0.00%,18.02%,653.16,122,1
13883,53,Thursday,W,Maine,B,Shannon Daley,GD-3,Training,Whole session,2:02:28,...,99.0,34.18,34.54,0.99,20.79%,6.76%,27.56%,683.77,122,1
13885,53,Thursday,W,Maine,M,Peyton Halsey,GD-3,Training,Whole session,2:02:28,...,65.0,29.62,28.14,1.05,14.02%,26.29%,40.31%,770.52,122,1
13892,53,Thursday,W,Maine,M,Alia Marshall,GD-3,Training,Whole session,2:02:28,...,57.0,25.48,26.50,0.96,16.59%,0.14%,16.73%,648.05,122,1


In [55]:
# Changing HR zones to minutes
train['HR_1'] = train['HR_1'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_2'] = train['HR_2'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_3'] = train['HR_3'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_4'] = train['HR_4'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_5'] = train['HR_5'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['Acute_HR_Sum'] = train['Acute_HR_Sum'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['Chronic_HR_Sum'] = train['Chronic_HR_Sum'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_28526/3473366246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HR_1'] = train['HR_1'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_28526/3473366246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HR_2'] = train['HR_2'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_28526/3473366246.py

,Week,Day,W_L,Game_Prep,Position,Athlete,Session_Name,Session,Phase_Name,Duration,...,Decel_Sum,Acute_Decel,Chronic_Decel,TSB_Decel,HR_4_Percent,HR_5_Percent,Percent_Above_80,TRIMP,Duration_(min),W_L_binary
13879,53,Thursday,W,Maine,F,Bente Baekers,GD-3,Training,Whole session,2:02:28,...,80.0,32.50,32.66,0.99,22.77%,7.09%,29.86%,699.08,122,1
13880,53,Thursday,W,Maine,B,Kayla Blas,GD-3,Training,Whole session,2:02:28,...,62.0,25.67,29.48,0.87,18.02%,0.00%,18.02%,653.16,122,1
13883,53,Thursday,W,Maine,B,Shannon Daley,GD-3,Training,Whole session,2:02:28,...,99.0,34.18,34.54,0.99,20.79%,6.76%,27.56%,683.77,122,1
13885,53,Thursday,W,Maine,M,Peyton Halsey,GD-3,Training,Whole session,2:02:28,...,65.0,29.62,28.14,1.05,14.02%,26.29%,40.31%,770.52,122,1
13892,53,Thursday,W,Maine,M,Alia Marshall,GD-3,Training,Whole session,2:02:28,...,57.0,25.48,26.50,0.96,16.59%,0.14%,16.73%,648.05,122,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,65,Sunday,L,NCAA_UNC,M,Alia Marshall,NCAA_UNC,Game,Whole session,2:26:00,...,109.0,30.74,31.71,0.97,37.04%,8.77%,45.81%,906.42,146,0
22261,65,Sunday,L,NCAA_UNC,M,Ana Medina Garcia,NCAA_UNC,Game,Whole session,2:26:00,...,111.0,29.15,27.79,1.05,32.74%,22.65%,55.39%,955.08,146,0
22262,65,Sunday,L,NCAA_UNC,M,Chloe Relford,NCAA_UNC,Game,Whole session,2:26:00,...,111.0,36.18,32.85,1.1,14.42%,22.09%,36.51%,900.33,146,0
22265,65,Sunday,L,NCAA_UNC,M,Lauren Wadas,NCAA_UNC,Game,Whole session,2:26:00,...,88.0,27.36,27.92,0.98,12.90%,8.63%,21.53%,851.67,146,0


In [57]:
train.corrwith(train.W_L_binary).sort_values(ascending = False)

W_L_binary         1.000000
Chronic_TD         0.089316
TLScore_Chronic    0.080723
HR_AVG             0.054697
HR_Avg_Per         0.054277
Recovery           0.050656
HR_3               0.044949
Chronic_HR_Sum     0.026896
Decel_2            0.023906
Acute_HIA          0.021393
HR_Max_Per         0.020642
HR_Max             0.019570
Chronic_HIA        0.015943
Speed_5            0.014181
TLScore_Acute      0.012378
Accel_4            0.010027
TLScore            0.007662
HIA                0.006607
Accel_3            0.005298
Accel_2            0.003867
HR_4               0.003810
Sprints            0.003068
Accel_1           -0.000331
HSD               -0.000598
Cardio_Load       -0.001973
Max_Speed         -0.006403
HR_5              -0.006763
HR_2              -0.007620
Speed_4           -0.009407
Calories          -0.010086
Decel_3           -0.015878
Acute_HR_Sum      -0.021777
Decel_Sum         -0.022557
TRIMP             -0.026036
Speed_3           -0.026278
HR_Min_Per        -0

In [58]:
## Below here I am playing around with shifting the dataframe

In [61]:
train.Athlete.value_counts()

Madeleine Zimmer     81
Kayla Blas           78
Chloe Relford        78
Peyton Halsey        77
Ana Medina Garcia    76
Lauren Wadas         76
Alia Marshall        75
Shannon Daley        68
Bente Baekers        65
Name: Athlete, dtype: int64

In [68]:
train.Date.value_counts()

2022-08-25    18
2022-09-27    18
2022-09-13    18
2022-09-07    17
2022-09-28    17
              ..
2022-09-08     8
2022-11-04     8
2022-09-22     7
2022-11-16     7
2022-10-25     7
Name: Date, Length: 69, dtype: int64